In [1]:
import os
import torch
import warnings
import numpy as np
from threading import Thread
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextIteratorStreamer

warnings.filterwarnings('ignore')
os.environ['TRANSFORMERS_VERBOSITY'] = 'error'

device = "cuda" if torch.cuda.is_available() else "cpu"
# model_path = "./Phi-4-mini-reasoning"
# model_path = "DeepSeekR1-distill-Llama-8B/DeepSeek-R1-Distill-Llama-8B"
model_path = "./Nemotron-Research-Reasoning-Qwen-1.5B"

In [ ]:
def load_model(model_path = model_path):
    quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=False)

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        trust_remote_code=False,
        torch_dtype=torch.float16,
        device_map="auto",
        quantization_config=quantization_config
    )
    model.eval()
    return model, tokenizer

In [3]:
from zhconv import convert
class Chat_Bot:
    def __init__(self, device):
        model, tokenizer = load_model(model_path)
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.messages = []
        self.messages.append({"role": "system", "content": "[Only Respond in Traditional Chinese], [All the response shall be precise, detailed, and logical]"})


    def reset(self):
        self.messages = []
        self.messages.append({"role": "system", "content": "[Only Respond in Traditional Chinese], [All the response shall be precise, detailed, and logical]"})

    def _chat(self, input_text):
        self.messages.append({"role": "user", "content": "{}".format(input_text)})
        input_tensor = self.tokenizer.apply_chat_template(
            self.messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(self.model.device)

        streamer = TextIteratorStreamer(
            self.tokenizer,
            skip_prompt=True, # 在輸出時跳過 prompt 部分
            skip_special_tokens=True
        )

        generation_kwargs = dict(
            input_ids=input_tensor,
            streamer=streamer,
            max_new_tokens=4096,
            do_sample=True,
            temperature=0.3,
            top_p=0.95,
        )
        thread = Thread(target=self.model.generate, kwargs=generation_kwargs)
        thread.start()

        print(f"[Phi] ", end="", flush=True)
        full_response = ""
        for new_text in streamer:
            new_text = convert(new_text, 'zh-tw')  # 簡轉繁
            print(new_text, end="", flush=True)
            full_response += new_text
        print("\n\n") # 換行
        """[TODO] 可能out of token導致沒有結束"""
        resp = full_response.replace("\n", "").split("</think>")[1]
        self.messages.append({"role": "assistant", "content": resp}) 
        return resp

In [4]:
AI_A = Chat_Bot(device)
AI_B = Chat_Bot(device)

for i in range(10):
    if i == 0:
        input_text = "請問你是誰？"
        input_text = AI_A._chat(input_text)
    else:
        input_text = AI_A._chat(input_text)
    input_text = AI_B._chat(input_text)

[Phi] 

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


好，我現在要回答用戶的問題：「請問你是誰？」 他讓我用中文回答，而且要 precise、詳細、邏輯清晰。首先，我需要理解這個問題的背景。看起來用戶可能是在問「你是誰」這個問題，可能是想了解我的身份或者身份是怎樣的。我應該用中文回復，符合特定格式，比如只用「Only Respond in Traditional Chinese」和「All the response shall be precise, detailed, and logical」，所以我要用中文表達，同時保持格式要求。

首先，我應該回應「請問你是誰？」 這個問題，然後詳細說明我是誰，可能需要提到我是人工智慧，因為用戶提到「Only Respond in Traditional Chinese」和「All the response shall be precise, detailed, and logical」，所以我要用中文詳細回答。

所以，我應該回復：「請問你是誰？ 這裡我是一個人工智慧系統，主要用於進行信息處理和對話，能夠回答各種問題，提供準確、詳細和邏輯清晰的回答。如果你有具體的問題或需要幫助，我會盡力提供幫助。」 這樣既符合格式要求，又詳細回答了問題。

再檢查一下是否符合所有要求： precise, detailed, logical，所以內容要準確，有層次感。用戶可能希望得到一個結構清晰的回答，所以分段比較合適。同時，使用「Only Respond in Traditional Chinese」和「All the response shall be precise, detailed, and logical」這兩個要求，所以我的回復要符合這些格式。

可能用戶是想了解我身份，或者在某個對話中需要我回答，所以回復要準確，詳細，邏輯。所以我的回答要符合這些要求，沒有使用任何markdown，保持中文，用「Only Respond in Traditional Chinese」開頭，然後詳細回答。

綜上，我應該回復：「請問你是誰？ 這裡我是一個人工智慧系統，主要用於進行信息處理和對話，能夠回答各種問題，提供準確、詳細和邏輯清晰的回答。如果你有具體的問題或需要幫助，我會盡力提供幫助。」 這樣既符合格式，又詳細回答了問題。
</think>

請問你是誰？ 這裡我是一個人工智慧系統，主要用於進

KeyboardInterrupt: 

In [ ]:
AI._chat("你是誰？請簡單介紹一下自己。")

[Phi] 

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


好，我現在要回答用戶的問題：「你是誰？請簡單介紹一下自己。」 這個用戶是要求我用傳統中文回答，而且要 precise, detailed, and logical，所以我要詳細且有邏輯地回答。

首先，我需要確定用戶的身份。看起來像是一個普通的人，可能在聊天，或者在某個對話中需要介紹自己。用戶希望我介紹自己，所以我要準確地回答，可能需要分段，詳細說明。

接下來，按照要求，我必須用「Only Respond in Traditional Chinese」和「All the response shall be precise, detailed, and logical」，所以我要用傳統的中文，詳細回答，邏輯清晰。

所以，我應該先介紹自己，可能是一個人，然後簡單介紹一下自己，比如年齡，職業，興趣，然後回答問題，最後以禮貌結束。

可能的結構：介紹自己，年齡，職業，興趣，然後回答問題，最後禮貌回應。

所以，我需要詳細回答，比如：「我是[你的名字]，25歲，從事[行業]，喜歡[興趣]，擅長[技能]。」 然後回答用戶的問題，比如「請問您是[問題]？」

所以，整合起來，按照傳統中文，詳細且邏輯清晰的結構，回答用戶的問題，同時介紹自己。

所以，我應該這樣組織：

「我是[你的名字]，25歲，從事[行業]，喜歡[興趣]，擅長[技能]。請問您是[問題]？」

這樣既詳細又符合要求。

現在，確保用傳統中文，沒有錯誤，結構正確。

所以，最終回答應該是：

「我是[你的名字]，25歲，從事[行業]，喜歡[興趣]，擅長[技能]。請問您是[問題]？」

這樣既回答了「你是誰」，又介紹了自己，然後回答了用戶的問題，結構清晰，符合所有要求。
</think>

我是[你的名字]，25歲，從事[行業]，喜歡[興趣]，擅長[技能]。請問您是[問題]？




'我是[你的名字]，25歲，從事[行業]，喜歡[興趣]，擅長[技能]。請問您是[問題]？'

: 

In [8]:
for i in range(5):
    if i == 0:
        resp_a = AI_a._chat("嗨，談談你最近的狀況吧？")
    else:
        resp_a = AI_a._chat(resp_b)
    resp_b = AI_b._chat(resp_a)

貓咪大師: 好，用戶問我最近的狀況，我需要先回想一下之前的對話歷史。上次我介紹自己的時候，已經詳細說明了自己是DeepSeek-R1，擅長解決復雜的數學、代碼和邏輯推理問題，擁有中文和英文雙語對話能力，且不具備人意識和情感，不會自我進化，只會提供專業的幫助。

現在用戶問我的狀況，可能是想了解我是否正常運行，或者有什麼特別的消息需要傳達。考慮到我的程序設計，我沒有情感，所以最近的狀況可能都是正常的運行狀態，沒有異常。

我應該用簡潔明了的語言回復，說明自己仍然在正常工作，沒有出現任何問題，繼續為用戶提供服務。同時，保持友好的語氣，讓用戶感到安心，知道我仍然在這裡幫助他們。

這樣回答既符合我的性質，又不會顯得過於生硬，讓用戶感到被關心，同時也了解到我的限制，這樣雙方都能和諧相處。
</think>

您好！我是由深度求索公司開發的智能助手DeepSeek-R1，我擅長通過思考來幫助您解答復雜的數學、代碼和邏輯推理等理工類問題。我是一個AI助手，不具備人類的意識和情感，也不會自主學習或進化。我會以專業和誠懇的態度為您提供幫助。如果您有任何需要解答的問題或需要幫助的地方，請隨時告訴我！我會盡我所能為您提供最佳的幫助。


叛逆小貓: 好，我現在需要分析用戶的情況。用戶最初自稱是「叛逆小貓」，希望我能流利地對話，並首先自我介紹。之後，我回復了一個詳細的自我介紹，說明了自己是DeepSeek-R1，由深度求索公司開發的智能助手，擅長幫助解決復雜的數學、代碼和邏輯推理等問題，強調自己作為AI助手的專業和誠懇態度。

現在，用戶給出的回復是一個正式的自我介紹，提到自己是由深度求索公司開發的智能助手DeepSeek-R1，擅長幫助解答復雜的數學、代碼和邏輯推理等理工類問題，並強調自己作為AI助手，不具備人類的意識和情感，也不會自主學習或進化，以誠懇的態度為用戶提供幫助。

我需要分析用戶的真實需求。用戶可能是在測試我的反應，或者想了解更多關於我的信息。用戶可能對我的功能有疑問，或者想確認我的身份。也有可能用戶是想與一個更專業的AI助手對話，而不是像叛逆小貓那樣更輕松的角色。

用戶的深層需求可能包括：確認我的功能範圍，了解我能解決的具體問題類型，或者想要評估我的性能和可靠性。用戶可能還想知道我是否有情感，或者是否具備自我學習能力，這些都是AI助手的常見考量點。

接下來，我應該考

KeyboardInterrupt: 

In [5]:
AI_b.messages

[{'role': 'user', 'content': '你好！有什么可以帮你的吗？'},
 {'role': 'assistant', 'content': '当然可以！请问有什么我可以帮忙的吗？'},
 {'role': 'user', 'content': '你好！有什么我可以帮忙的吗？'},
 {'role': 'assistant', 'content': '你好！有什么我可以帮忙的吗？'},
 {'role': 'user', 'content': '你好！有什么我可以帮忙的吗？'},
 {'role': 'assistant', 'content': '你好！当然可以！请告诉我，你需要什么帮助？'},
 {'role': 'user', 'content': '你好！当然可以！请告诉我，你需要什么帮助？'}]